# Notebook Setup

In [39]:
import pandas as pd
import numpy as np
import string
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
products = pd.read_csv('C:/Users/212313601/Documents/Coursera/Machine Learning Specialization - Classification/amazon_baby.csv')

In [3]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


# Cleaning

In [4]:
def remove_punctuation(text):
    import string
    return text.translate(string.punctuation)

In [5]:
products.review.isnull().sum()

829

In [6]:
products.review.fillna('',inplace=True)

In [7]:
products.review.isnull().sum()

0

In [8]:
products['review_clean'] = products['review'].apply(remove_punctuation)

# Test/Train Split - <code>train_data</code> and <code>test_data</code> 

In [22]:
test_indic = open('C:/Users/212313601/Documents/Coursera/Machine Learning Specialization - Classification/test_indic.txt')
test_list=[]
for line in t:
    t = [int(x.strip()) for x in line.split(',')]
    test_list.append(t[0])

In [38]:
test_data = products.iloc[test_list]
test_data.head()

,name,review,rating,review_clean
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...
14,Nature's Lullabies First Year Sticker Calendar,"Space for monthly photos, info and a lot of us...",5,"Space for monthly photos, info and a lot of us..."
18,Nature's Lullabies Second Year Sticker Calendar,I completed a calendar for my son's first year...,4,I completed a calendar for my son's first year...
24,Nature's Lullabies Second Year Sticker Calendar,Wife loves this calender. Comes with a lot of ...,5,Wife loves this calender. Comes with a lot of ...


In [36]:
train_indic = open('C:/Users/212313601/Documents/Coursera/Machine Learning Specialization - Classification/test_indic.txt')
train_list=[]
for line in t:
    t = [int(x.strip()) for x in line.split(',')]
    train_list.append(t[0])

In [37]:
train_data = products.iloc[train_list]
train_data.head()

,name,review,rating,review_clean
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...
14,Nature's Lullabies First Year Sticker Calendar,"Space for monthly photos, info and a lot of us...",5,"Space for monthly photos, info and a lot of us..."
18,Nature's Lullabies Second Year Sticker Calendar,I completed a calendar for my son's first year...,4,I completed a calendar for my son's first year...
24,Nature's Lullabies Second Year Sticker Calendar,Wife loves this calender. Comes with a lot of ...,5,Wife loves this calender. Comes with a lot of ...


# Build the word count vector for each review

In [ ]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')